TLDR: 
1. Purpose: A demo of **tool use** to do a simple math problem to calculate the cosine of 8
2. Model name: `anthropic--claude-3-opus`
3. Model Provider: Amazon Bedrock
4. Only one tool function is used
5. Reference: [Amazon Bedrock](https://community.aws/content/2hW5367isgQOkkXLYjp4JB3Pe16/intro-to-tool-use-with-the-amazon-bedrock-converse-api?lang=en)

In [10]:
import math
from pprint import pprint

from gen_ai_hub.proxy.native.amazon.clients import Session

model_name = 'anthropic--claude-3-opus'
bedrock = Session().client(model_name=model_name)

In [11]:
tool_list = [
    {
        "toolSpec": {
            "name": "cosine",
            "description": "Calculate the cosine of x",
            "inputSchema": {
                "json": {
                    "type": "object", 
                    "properties": {
                        "x": {
                            "type": "number", 
                            "description": "The number to pass to the function"
                        }
                    },
                    "required": ["x"]
                }
            }
        }
    }
]

In [12]:
def call_bedrock(message_list, tool_list):
    bedrock = Session().client(model_name=model_name)
    response = bedrock.converse(
        messages = message_list, 
        inferenceConfig = {
            "maxTokens": 4096, 
            "temperature": 0.0, 
        },
        toolConfig={
            "tools": tool_list
        }
    )
    return response

In [13]:
message_list = []
initial_message = {
    "role": "user", 
    "content": [
        {"text": "What is the cosine of 8?"}
    ]
}
message_list.append(initial_message)
print(message_list)

[{'role': 'user', 'content': [{'text': 'What is the cosine of 8?'}]}]


In [14]:
# Call 1: User -> Assistant
response = call_bedrock(message_list, tool_list)

In [15]:
pprint(response)
response_message = response['output']['message']
response_content_blocks = response_message['content']
# add the response to the conversation history
message_list.append(response_message)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '688',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 02 Dec 2024 18:04:41 GMT'},
                      'HTTPStatusCode': 200,
                      'RetryAttempts': 0},
 'metrics': {'latencyMs': 6917},
 'output': {'message': {'content': [{'text': '<thinking>\n'
                                             'The user is asking for the '
                                             'cosine of the number 8. To '
                                             'answer this, the relevant tool '
                                             'is:\n'
                                             '\n'
                                             'cosine:\n'
                                             '- Description: Calculate the '
                                             'cosine of x\n'
                                             '- Required parameter: \n'
   

In [17]:
follow_up_content_blocks = []

for content_block in response_content_blocks:
    if 'toolUse' in content_block:
        tool_use_block = content_block['toolUse']
        tool_use_id = tool_use_block['toolUseId']
        tool_use_name = tool_use_block['name']

        if tool_use_name == 'cosine':
            x = tool_use_block['input']['x']
            tool_result_value = math.cos(x)

            follow_up_content_blocks.append({
                    "toolResult": {
                        'toolUseId': tool_use_id,
                        'content':[
                            {
                                "json": {
                                    "result": tool_result_value
                                }
                            }
                        ]
                    }
                })

if len(follow_up_content_blocks) > 0:

    follow_up_message = {
        "role": "user", 
        "content": follow_up_content_blocks,
    }

    message_list.append(follow_up_message)

pprint(message_list)

[{'content': [{'text': 'What is the cosine of 8?'}], 'role': 'user'},
 {'content': [{'text': '<thinking>\n'
                       'The user is asking for the cosine of the number 8. To '
                       'answer this, the relevant tool is:\n'
                       '\n'
                       'cosine:\n'
                       '- Description: Calculate the cosine of x\n'
                       '- Required parameter: \n'
                       '  - x (number): The number to pass to the function\n'
                       '\n'
                       'The user has directly provided the required parameter '
                       'x, with a value of 8. Since all required parameters '
                       'are available, we can proceed with calling the cosine '
                       'tool.\n'
                       '</thinking>'},
              {'toolUse': {'input': {'x': 8},
                           'name': 'cosine',
                           'toolUseId': 'tooluse_nHfKR9AAQaG4G

In [18]:
# Call 2: User -> Assistant
response = call_bedrock(message_list, tool_list)

In [19]:
response_message = response['output']['message']
response_message_content = response_message['content']
response_stop_reason = response['stopReason']
if response_stop_reason == 'end_turn':
    result = response_message_content[0]['text']
    print(f'The result is: {result}')

The result is: The cosine of 8 is -0.14550003380861354.
